# Project Titan — Treino Híbrido YOLOv8 (Colab GPU)

Este notebook executa o treino híbrido do modelo de detecção:
- **synthetic_v3/** (8500 train / 1500 val) — PPPoker-realistic com gold borders + domain randomization
- **synthetic/** (1700 train / 300 val) — cartas sintéticas simples (classes 0-51)
- **titan_cards/** (177 train / 33 val) — botões, pot, stack reais (classes 52-61)

**Total: ~10.377 train / ~1.833 val — 62 classes**

### Pré-requisitos

1. Subir `titan_pacotes.zip` para Google Drive em `Titan_Training/`
2. Selecionar runtime **GPU T4** (Runtime → Change runtime type → T4)
3. Executar todas as células em ordem

## 1. Verificar GPU e montar Drive

In [ ]:
# Verificar GPU disponível
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Extrair projeto e configurar ambiente

In [ ]:
import os, shutil, zipfile

# ── Configuração ──
ZIP_PATH = '/content/drive/MyDrive/Titan_Training/titan_pacotes.zip'
WORKSPACE = '/content/workspace'
PROJECT   = f'{WORKSPACE}/project_titan'
DATASETS  = f'{WORKSPACE}/datasets'      # Raiz dos datasets
RUN_NAME  = 'titan_v7_hybrid'             # ← Mude aqui para cada run

# ── Limpar workspace anterior ──
if os.path.exists(WORKSPACE):
    shutil.rmtree(WORKSPACE)
os.makedirs(WORKSPACE, exist_ok=True)

# ── Extrair zip ──
print(f'Extraindo {ZIP_PATH}...')
with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(WORKSPACE)
print('Extração concluída.')

# ── Corrigir estrutura se datasets ficou dentro de project_titan/ ──
nested_datasets = os.path.join(PROJECT, 'datasets')
if os.path.exists(nested_datasets) and not os.path.exists(DATASETS):
    print('Movendo datasets/ para fora de project_titan/...')
    shutil.move(nested_datasets, DATASETS)
    print('OK')

# ── Verificar estrutura ──
checks = {
    'data.yaml':        f'{PROJECT}/training/data.yaml',
    'train_yolo.py':    f'{PROJECT}/training/train_yolo.py',
    'synthetic_v3/train':f'{DATASETS}/synthetic_v3/images/train',
    'synthetic_v3/val': f'{DATASETS}/synthetic_v3/images/val',
    'synthetic/train':  f'{DATASETS}/synthetic/images/train',
    'synthetic/val':    f'{DATASETS}/synthetic/images/val',
    'titan_cards/train':f'{DATASETS}/titan_cards/images/train',
    'titan_cards/val':  f'{DATASETS}/titan_cards/images/val',
}
all_ok = True
for name, path in checks.items():
    exists = os.path.exists(path)
    icon = '✅' if exists else '❌'
    extra = ''
    if exists and os.path.isdir(path):
        count = len(os.listdir(path))
        extra = f' ({count} arquivos)'
    print(f'  {icon} {name}: {path}{extra}')
    if not exists:
        all_ok = False

if all_ok:
    print('\n✅ Estrutura completa — pronto para treinar!')

else:
    print('\n❌ Estrutura incompleta — verifique o zip.')

## 3. Corrigir data.yaml para caminhos do Colab

In [ ]:
import yaml

data_yaml_path = f'{PROJECT}/training/data.yaml'

with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Forçar path absoluto do Colab para a raiz dos datasets
data['path'] = DATASETS

# Garantir que train/val são listas (treino híbrido — 3 fontes)
data['train'] = [
    'synthetic_v3/images/train',
    'synthetic/images/train',
    'titan_cards/images/train',
]
data['val'] = [
    'synthetic_v3/images/val',
    'synthetic/images/val',
    'titan_cards/images/val',
]

with open(data_yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False, sort_keys=False)

print(f'data.yaml atualizado:')
print(f'  path:  {data["path"]}')
print(f'  train: {data["train"]}')
print(f'  val:   {data["val"]}')

print(f'  nc:    {data["nc"]}')
print(f'  classes 52-61: {[data["names"][i] for i in range(52, 62)]}')

## 4. Instalar dependências

In [ ]:
!pip install -q ultralytics

## 5. Treinar modelo híbrido

In [ ]:
import os
os.chdir(WORKSPACE)

# Usa caminho absoluto no --data para evitar qualquer ambiguidade
!python -u {PROJECT}/training/train_yolo.py \
    --data {PROJECT}/training/data.yaml \
    --epochs 100 \
    --model yolov8n.pt \
    --batch 16 \
    --name {RUN_NAME}

## 6. Salvar resultados no Google Drive

In [ ]:
import os, shutil, glob

# Encontrar a pasta de resultados (pode variar)
search_patterns = [
    f'{WORKSPACE}/runs/{RUN_NAME}',
    f'{WORKSPACE}/runs/detect/{RUN_NAME}',
    f'{WORKSPACE}/runs/runs/{RUN_NAME}',
]
run_dir = None
for p in search_patterns:
    if os.path.exists(p):
        run_dir = p
        break

# Fallback: buscar best.pt recursivamente
if run_dir is None:
    found = glob.glob(f'{WORKSPACE}/runs/**/weights/best.pt', recursive=True)
    if found:
        run_dir = os.path.dirname(os.path.dirname(found[-1]))
        print(f'Encontrado via busca: {run_dir}')

if run_dir is None:
    print('❌ Pasta de resultados não encontrada!')
    print('Conteúdo de runs/:')
    !find {WORKSPACE}/runs -type f -name "*.pt" 2>/dev/null
else:
    DRIVE_DST = f'/content/drive/MyDrive/Titan_Training/{RUN_NAME}'
    os.makedirs(DRIVE_DST, exist_ok=True)
    shutil.copytree(run_dir, DRIVE_DST, dirs_exist_ok=True)
    
    # Verificar pesos salvos
    best_pt = os.path.join(DRIVE_DST, 'weights', 'best.pt')
    last_pt = os.path.join(DRIVE_DST, 'weights', 'last.pt')
    print(f'\n✅ Resultados salvos em: {DRIVE_DST}')
    if os.path.exists(best_pt):
        size_mb = os.path.getsize(best_pt) / 1024 / 1024
        print(f'   best.pt: {size_mb:.1f} MB')
    if os.path.exists(last_pt):
        size_mb = os.path.getsize(last_pt) / 1024 / 1024
        print(f'   last.pt: {size_mb:.1f} MB')
    print(f'\nPara usar localmente:')
    print(f'  Copie best.pt para: project_titan/models/titan_v7_hybrid.pt')

## 7. (Opcional) Resumir treino interrompido

In [ ]:
# Descomente e execute apenas se o treino foi interrompido:

# RESUME_PT = f'/content/drive/MyDrive/Titan_Training/{RUN_NAME}/weights/last.pt'
# 
# os.chdir(WORKSPACE)
# !python -u {PROJECT}/training/train_yolo.py \
#     --resume {RESUME_PT} \
#     --name {RUN_NAME}

## 8. Métricas finais

In [ ]:
from IPython.display import Image, display
import glob

# Mostrar gráficos de treino
plots = glob.glob(f'{WORKSPACE}/runs/**/{RUN_NAME}/results.png', recursive=True)
if not plots:
    plots = glob.glob(f'{WORKSPACE}/runs/**/results.png', recursive=True)

if plots:
    print(f'Resultados de: {plots[-1]}')
    display(Image(filename=plots[-1], width=900))
else:
    print('Gráfico results.png não encontrado.')

# Mostrar confusion matrix
cm = glob.glob(f'{WORKSPACE}/runs/**/{RUN_NAME}/confusion_matrix.png', recursive=True)
if not cm:
    cm = glob.glob(f'{WORKSPACE}/runs/**/confusion_matrix.png', recursive=True)

if cm:
    print(f'\nConfusion Matrix: {cm[-1]}')
    display(Image(filename=cm[-1], width=900))